## BETTER WAY 27 - 비공개 속성보다는 공개 속성을 사용하자
## Item27 - Prefer Public Attributes Over Private Ones

파이썬에는 클래스 속성의 가시성(visibility)이 공개(public)와 비공개(private) 두 유형밖에 없다. <br />
아래의 예제 클래스를 보자

In [2]:
class MyObject(object):
    def __init__(self):
        self.public_field = 5
        self.__private_field = 10
        
    def get_private_field(self):
        return self.__private_field

공개 속성은 어디에서든 객체에 점 연산자(`.`)를 사용하여 접근할 수 있다.

In [3]:
foo = MyObject()
assert foo.public_field == 5

비공개 필드는 속성 이름 앞에 밑줄 두 개를 붙여 지정한다.([Better way 02 참고](https://github.com/ExcelsiorCJH/Study/blob/master/Python/Effective-python/Chap01-Pythonic_Thinking/Item02-Follow_the_PEP8_Style_Guide.ipynb)) <br /> 
같은 클래스에 속한 메소드에서는 비공개 필드에 직접 접근할 수 있다.

In [4]:
print(foo.get_private_field())

10


하지만 클래스 밖에서 직접 비공개 필드에 접근할 경우 에러가 난다.

In [5]:
print(foo.__private_field)

AttributeError: 'MyObject' object has no attribute '__private_field'

클래스메소드(`@classmethod`)도 같은 `class` 블록에 선언되어 있으므로 비공개 속성에 접근할 수 있다.

In [7]:
class MyOtherObject(object):
    def __init__(self):
        self.__private_field = 71
        
    @classmethod
    def get_private_field_of_instance(cls, instance):
        return instance.__private_field
    
    
bar = MyOtherObject()
print(MyOtherObject.get_private_field_of_instance(bar))

71


비공개 필드는 상속받는 서브클래스에서는 부모 클래스의 비공개 필드에 접근할 수 없다.

In [8]:
class MyParentObject(object):
    def __init__(self):
        self.__private_field = 71
        
        
class MyChildObject(MyParentObject):
    def get_private_field(self):
        return self.__private_field
    

baz = MyChildObject()
baz.get_private_field()

AttributeError: 'MyChildObject' object has no attribute '_MyChildObject__private_field'

비공개 속성의 동작은 간단하게 속성 이름을 바꾸는 방식으로 구현된다. <br />
파이썬 컴파일러는 위의`MyChildObject.get_private_field` 메소드에서 비공개 속성에 접근하는 코드를 발견하면, `__private_field` 를 `_MyChildObject__private_field` 에 접근하는 코드로 변환한다.

위의 예제에서는 `__private_field` 가 `MyParentObject.__init__`에만 정의 되어 있으므로 비공개 속성의 실제 이름은 `_MyParentObject__private_field` 가 된다. <br />
따라서, 자식 클래스에서 부모 클래스의 비공개 속성에 접근하는 동작은 단순히 속성 이름이 바껴 일치하지 않아서 에러가 나는 것이다. 아래의 예제를 보자.

In [9]:
print(baz._MyParentObject__private_field)

71


아래의 코드처럼 객체의 속성 딕셔너리를 살펴보면 실제로 비공개 속성의 이름이 변환되어 저장되어 있음을 알 수 있다.

In [10]:
print(baz.__dict__)

{'_MyParentObject__private_field': 71}


파이썬에서 비공개 속성 문법이 가시성을 엄격하게 강제하지 않는 이유는 무엇일까? 교재에서는 아래와 같은 이유 때문이라고 한다.
> 가장 간단한 답은 파이썬에서 자주 인용되는 “우리 모두 성인이라는 사실에 동의합니다”라는 좌우명에 있다. 파이썬 프로그래머들은 개방으로 얻는 장점이 폐쇄로 얻는 단점보다 크다고 믿는다.

이러한 이유로 파이썬 프로그래머들은 무분별하게 객체의 내부에 접근하는 위험을 최소화하려고 스타일 가이드 ([Better way 02 참고](https://github.com/ExcelsiorCJH/Study/blob/master/Python/Effective-python/Chap01-Pythonic_Thinking/Item02-Follow_the_PEP8_Style_Guide.ipynb))에 정의된 Naming 관례를 따른다.

`_protected_field` 처럼 앞에 밑줄 한 개를 붙인 필드는 보호(protected) 필드로 취급해서 클래스의 외부 사용자들이 신중하게 다뤄야 함을 의미한다.

## 정리
- 파이썬 컴파일러는 비공개 속성을 엄격하게 강요하지 않는다.
- 서브클래스가 내부 API와 속성에 접근하지 못하게 막기보다는 처음부터 내부 API와 속성으로 더 많은 일을 할 수 있게 설계하자.
- 비공개 속성에 대한 접근을 강제로 제어하지 말고 보호 필드를 문서화해서 서브클래스에 필요한 지침을 제공하자.
- 직접 제어할 수 없는 서브클래스와 이름이 충돌하지 않게 할 때만 비공개 속성을 사용하는 방안을 고려하자.